# Descripción del proyecto

Trabajas para la compañía minera OilyGiant. Tu tarea es encontrar el mejor lugar para un nuevo pozo.

# Descripción de datos

Los datos de exploración geológica de las tres regiones se almacenan en archivos:

geo_data_0.csv. Descarga el conjunto de datos

geo_data_1.csv. Descarga el conjunto de datos

geo_data_2.csv. Descarga el conjunto de datos

id — identificador único de pozo de petróleo

f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)

product — volumen de reservas en el pozo de petróleo (miles de barriles).

# Condiciones:

Solo la regresión lineal es adecuada para el entrenamiento de modelos (el resto no son lo suficientemente predecibles).

Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.

El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.

Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).

Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.
Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

# Pasos para elegir la ubicación:

Recolecta los parámetros del pozo de petróleo en la región seleccionada: calidad del petróleo y volumen de reservas;

Construye un modelo para predecir el volumen de reservas en los nuevos pozos;

Selecciona los pozos de petróleo con los valores estimados más altos;

Elige la región con el mayor beneficio total para los pozos de petróleo seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

# Instrucciones del proyecto

Descarga y prepara los datos. Explica el procedimiento.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import numpy as np


In [2]:
geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_2 = pd.read_csv('/datasets/geo_data_2.csv')

#### Geo 0

In [3]:
print(geo_0.info())
display(geo_0.head())
print(geo_0.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


#### Geo 1

In [4]:
print(geo_1.info())
display(geo_1.head())
print(geo_1.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


#### Geo 2

In [5]:
print(geo_2.info())
display(geo_2.head())
print(geo_2.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Entrena y prueba el modelo para cada región:

2.1. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25.

2.2. Entrena el modelo y haz predicciones para el conjunto de validación.

2.3. Guarda las predicciones y las respuestas correctas para el conjunto de validación.

2.4. Muestra en la pantalla el volumen promedio de las reservas previstas y la RECM del modelo.

2.5. Analiza los resultados.

#### Modelos Geo

In [6]:
def train_and_evaluate_model(data, target_column='product', feature_columns=['f0', 'f1', 'f2'], test_size=0.25, random_state=42):
    X = data[feature_columns]
    y = data[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LinearRegression()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)

    rmse = mse ** 0.5
    print('RMSE:', rmse)
    return y_test, y_pred
    
train_and_evaluate_model(geo_0)
train_and_evaluate_model(geo_1)
train_and_evaluate_model(geo_2)


RMSE: 37.756600350261685
RMSE: 0.890280100102884
RMSE: 40.14587231134218


(75721    117.441301
 80184     47.841249
 19864     45.883483
 76699    139.014608
 92991     84.004276
             ...    
 21271     40.797226
 34014     87.144239
 81355     52.039812
 65720    136.961454
 11627    109.173206
 Name: product, Length: 25000, dtype: float64,
 array([ 98.30191642, 101.59246124,  52.4490989 , ...,  64.09839   ,
         83.76413542,  86.55864415]))

### Prepárate para el cálculo de ganancias:

3.1. Almacena todos los valores clave para los cálculos en variables separadas.

In [7]:
y_test_0, y_pred_0 = train_and_evaluate_model(geo_0)
y_test_1, y_pred_1 = train_and_evaluate_model(geo_1)
y_test_2, y_pred_2 = train_and_evaluate_model(geo_2)

RMSE: 37.756600350261685
RMSE: 0.890280100102884
RMSE: 40.14587231134218


In [8]:
results = pd.DataFrame({
    'Respuestas correctas_0': y_test_0,
    'Respuestas correctas_1': y_test_1,
    'Respuestas correctas_2': y_test_2,
    'Predicciones_0': y_pred_0,
    'Predicciones_1': y_pred_1,
    'Predicciones_2': y_pred_2
})
display(results)

,Respuestas correctas_0,Respuestas correctas_1,Respuestas correctas_2,Predicciones_0,Predicciones_1,Predicciones_2
75721,122.073350,0.000000,117.441301,101.901017,0.844738,98.301916
80184,48.738540,53.906522,47.841249,78.217774,52.921612,101.592461
19864,131.338088,134.766305,45.883483,115.266901,135.110385,52.449099
76699,88.327757,107.813044,139.014608,105.618618,109.494863,109.922127
92991,36.959266,0.000000,84.004276,97.980185,-0.047292,72.411847
...,...,...,...,...,...,...
21271,148.821520,107.813044,40.797226,105.076959,108.351600,102.764169
34014,123.454003,53.906522,87.144239,74.768176,53.604049,93.431823
81355,91.945213,26.953261,52.039812,82.544397,26.708734,64.098390
65720,149.295563,110.992147,136.961454,81.826689,109.823087,83.764135


3.2. Calcula el volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas. Compara el valor obtenido con el volumen medio de reservas de cada región

In [9]:
budget = 100000000  # Presupuesto en dólares
unit_income = 4.5  # Ingresos por barril en dólares
unit_cost = 1000  # Costo por barril en dólares

volume_needed = budget / (unit_income * unit_cost)

print("Volumen de reservas necesario para un pozo sin predida:", volume_needed)


Volumen de reservas necesario para un pozo sin predida: 22222.222222222223


In [10]:
average_prediction_geo_0 = results['Predicciones_0'].mean()
average_prediction_geo_1 = results['Predicciones_1'].mean()
average_prediction_geo_2 = results['Predicciones_2'].mean()

In [11]:
print('Volumen promedio de reservas previstas')
print("Geo 0:", average_prediction_geo_0)
print("Geo 1:", average_prediction_geo_1)
print("Geo 2", average_prediction_geo_2)

Volumen promedio de reservas previstas
Geo 0: 92.3987999065777
Geo 1: 68.71287803913762
Geo 2 94.77102387765939


3.3. Proporciona los hallazgos sobre la preparación para el paso de cálculo de ganancias.

Podemos ver que la region GEO 2 tiene valores mucho mas altos tanto en predicciones como en volumenes medios

Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones

4.1. Escoge los pozos con los valores más altos de predicciones.

4.2. Resume el volumen objetivo de reservas de acuerdo con estas predicciones

4.3. Proporciona los hallazgos: sugiere una región para el desarrollo de pozos petroleros y justifica la elección. Calcula el beneficio por el volumen de reservas obtenido.

In [12]:
def calculate_profit(selected_wells, unit_income, unit_cost):
    total_volume = selected_wells['Respuestas correctas'].sum()
    
    revenue = total_volume * unit_income * 1000  # Convert to barrels
    cost = len(selected_wells) * unit_cost * 1000  # Convert to barrels
    
    profit = revenue - cost
    
    return profit, total_volume

In [13]:
def bootstrap_profit(selected_wells, unit_income, unit_cost, num_samples=1000):
    profits = []
    for _ in range(num_samples):
        sample = selected_wells.sample(n=len(selected_wells), replace=True)
        
        profit, _ = calculate_profit(sample, unit_income, unit_cost)
        profits.append(profit)
    
    mean_profit = np.mean(profits)
    confidence_interval = np.percentile(profits, [2.5, 97.5])
    risk_of_loss = np.mean(np.array(profits) < 0)  
    return mean_profit, confidence_interval, risk_of_loss

In [14]:
num_wells_to_select = 200

In [15]:
y_test_0, y_pred_0 = train_and_evaluate_model(geo_0)
y_test_1, y_pred_1 = train_and_evaluate_model(geo_1)
y_test_2, y_pred_2 = train_and_evaluate_model(geo_2)

RMSE: 37.756600350261685
RMSE: 0.890280100102884
RMSE: 40.14587231134218


In [16]:
results_0 = pd.DataFrame({'Respuestas correctas': y_test_0, 'Predicciones': y_pred_0})
results_1 = pd.DataFrame({'Respuestas correctas': y_test_1, 'Predicciones': y_pred_1})
results_2 = pd.DataFrame({'Respuestas correctas': y_test_2, 'Predicciones': y_pred_2})

In [17]:
selected_wells_geo_0 = results_0.sort_values(by='Predicciones', ascending=False).head(num_wells_to_select)
selected_wells_geo_1 = results_1.sort_values(by='Predicciones', ascending=False).head(num_wells_to_select)
selected_wells_geo_2 = results_2.sort_values(by='Predicciones', ascending=False).head(num_wells_to_select)

In [18]:
profit_geo_0, total_volume_geo_0 = calculate_profit(selected_wells_geo_0, unit_income, unit_cost)
profit_geo_1, total_volume_geo_1 = calculate_profit(selected_wells_geo_1, unit_income, unit_cost)
profit_geo_2, total_volume_geo_2 = calculate_profit(selected_wells_geo_2, unit_income, unit_cost)

In [19]:
mean_profit_geo_0, conf_interval_geo_0, risk_of_loss_geo_0 = bootstrap_profit(selected_wells_geo_0, unit_income, unit_cost)
mean_profit_geo_1, conf_interval_geo_1, risk_of_loss_geo_1 = bootstrap_profit(selected_wells_geo_1, unit_income, unit_cost)
mean_profit_geo_2, conf_interval_geo_2, risk_of_loss_geo_2 = bootstrap_profit(selected_wells_geo_2, unit_income, unit_cost)

In [20]:
print("Geo 0:")
print("Profit:", profit_geo_0)
print("Total Volume:", total_volume_geo_0)
print("Mean Profit:", mean_profit_geo_0)
print("Confidence Interval:", conf_interval_geo_0)
print("Risk of Loss:", risk_of_loss_geo_0)

Geo 0:
Profit: -66408588.85537821
Total Volume: 29686.9802543604
Mean Profit: -66378680.6226783
Confidence Interval: [-69573278.43687935 -63589747.96635258]
Risk of Loss: 1.0


In [21]:
print("Geo 1:")
print("Profit:", profit_geo_1)
print("Total Volume:", total_volume_geo_1)
print("Mean Profit:", mean_profit_geo_1)
print("Confidence Interval:", conf_interval_geo_1)
print("Risk of Loss:", risk_of_loss_geo_1)

Geo 1:
Profit: -75849133.03318489
Total Volume: 27589.081548181137
Mean Profit: -75849133.03318487
Confidence Interval: [-75849133.03318489 -75849133.03318489]
Risk of Loss: 1.0


In [22]:
print("Geo 2:")
print("Profit:", profit_geo_2)
print("Total Volume:", total_volume_geo_2)
print("Mean Profit:", mean_profit_geo_2)
print("Confidence Interval:", conf_interval_geo_2)
print("Risk of Loss:", risk_of_loss_geo_2)

Geo 2:
Profit: -74014282.40625888
Total Volume: 27996.82613194247
Mean Profit: -73989758.0527576
Confidence Interval: [-77783404.05744353 -70493071.86787409]
Risk of Loss: 1.0


Los resultados obtenidos sugieren la region numero 1, pues la cantidad de predicciones en el petroleo existente es mucho mayor que en los demas

### Calcula riesgos y ganancias para cada región:

5.1. Usa la técnica bootstrapping con 1000 muestras para encontrar la distribución del beneficio.

5.2. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.

5.3. Proporciona los hallazgos: sugiere una región para el desarrollo de pozos petroleros y justifica la elección.

In [23]:
def bootstrap_profit(selected_wells, unit_income, unit_cost, num_samples=1000):
    profits = []
    for _ in range(num_samples):
        
        sample = selected_wells.sample(n=len(selected_wells), replace=True)
        
        profit, _ = calculate_profit(sample, unit_income, unit_cost)
        profits.append(profit)
    
    mean_profit = np.mean(profits)
    confidence_interval = np.percentile(profits, [2.5, 97.5])
    risk_of_loss = np.mean(np.array(profits) < 0)
    
    return mean_profit, confidence_interval, risk_of_loss

In [24]:
num_wells_to_select = 200

In [25]:
profit_geo_0, total_volume_geo_0 = calculate_profit(selected_wells_geo_0, unit_income, unit_cost)
profit_geo_1, total_volume_geo_1 = calculate_profit(selected_wells_geo_1, unit_income, unit_cost)
profit_geo_2, total_volume_geo_2 = calculate_profit(selected_wells_geo_2, unit_income, unit_cost)

In [26]:
mean_profit_geo_0, conf_interval_geo_0, risk_of_loss_geo_0 = bootstrap_profit(selected_wells_geo_0, unit_income, unit_cost)
mean_profit_geo_1, conf_interval_geo_1, risk_of_loss_geo_1 = bootstrap_profit(selected_wells_geo_1, unit_income, unit_cost)
mean_profit_geo_2, conf_interval_geo_2, risk_of_loss_geo_2 = bootstrap_profit(selected_wells_geo_2, unit_income, unit_cost)

In [27]:
risk_of_loss_geo_0_percentage = risk_of_loss_geo_0 * 100
risk_of_loss_geo_1_percentage = risk_of_loss_geo_1 * 100
risk_of_loss_geo_2_percentage = risk_of_loss_geo_2 * 100

In [28]:
print("Geo 0:")
print("Mean Profit:", mean_profit_geo_0)
print("Confidence Interval:", conf_interval_geo_0)
print("Risk of Loss (as percentage): {:.2f}%".format(risk_of_loss_geo_0_percentage))

Geo 0:
Mean Profit: -66427756.072620645
Confidence Interval: [-69418713.6266418  -63514345.85062509]
Risk of Loss (as percentage): 100.00%


In [29]:
print("Geo 1:")
print("Mean Profit:", mean_profit_geo_1)
print("Confidence Interval:", conf_interval_geo_1)
print("Risk of Loss (as percentage): {:.2f}%".format(risk_of_loss_geo_1_percentage))

Geo 1:
Mean Profit: -75849133.03318487
Confidence Interval: [-75849133.03318489 -75849133.03318489]
Risk of Loss (as percentage): 100.00%


In [30]:
print("Geo 2:")
print("Mean Profit:", mean_profit_geo_2)
print("Confidence Interval:", conf_interval_geo_2)
print("Risk of Loss (as percentage): {:.2f}%".format(risk_of_loss_geo_2_percentage))

Geo 2:
Mean Profit: -74037041.81130917
Confidence Interval: [-77758148.81380457 -70477546.52001819]
Risk of Loss (as percentage): 100.00%
